In [1]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
from ipywidgets import interact, widgets
from pathlib import Path

from os import listdir
from os.path import isfile, join


p = Path.home()
p = p/'OneDrive'/'Documentos'/'Python'/'Results'/'CSV'/'FTIR'/'Espectros Carlos Henrique'

In [2]:
filenames = [f for f in listdir(p) if isfile(join(p, f))]
frames = dict()
for filename in filenames:
        filepath = p/filename
        try:
            frames[filepath.name] = (pd.read_csv(filepath))
            frames[filepath.name].columns = ['Wavenumber','%Absorbance']
            frames[filepath.name]['%Transmitance'] = 1-frames[filepath.name]['%Absorbance']
        except FileNotFoundError:
            pass

In [3]:
def plot_ftir(key, kind):
    if kind == 'absorbance':
        fig  = frames[key].plot(x = 'Wavenumber', y='%Absorbance',kind='line')
    elif kind == 'transmitance':
        fig  = frames[key].plot(x = 'Wavenumber', y='%Transmitance',kind='line')
    fig.update_layout(xaxis_range = [4000,500], yaxis_tickformat = '%')
    return fig

In [4]:
interact(plot_ftir,
         key = list(frames.keys()),
         kind = ['absorbance', 'transmitance']
        )

interactive(children=(Dropdown(description='key', options=('A P1 LA.csv', 'A P1 LB.csv', 'B P1 LA.csv', 'B P1 …

<function __main__.plot_ftir(key, kind)>

In [5]:
for key, spec in frames.items():
    for kind in ['absorbance', 'transmitance']:
        path=p/'{}_{}.html'.format(key[:-4], kind)
        plot_ftir(key, kind).write_html(str(path))

In [15]:
fig = go.Figure()
for key, frame in frames.items():
    df = frame
    if key == list(frames.keys())[-1]:
        visible = True
    else:
        visible = 'legendonly'
    fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], name = key, visible = visible, legendgroup = key))
for key, frame in frames.items():
    df = frame
    fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Absorbance'], name = key, visible = False, legendgroup = key))

fig.update_layout(xaxis_range = [4000,500],
                  xaxis_title_text = 'Número de onda (1/cm)',
                  yaxis_title_text = 'Transmitância (u.a.)',
                  legend_title_text = 'Clique para mostrar!',
                  title_text = 'Espectros de Infra-vermelho',
                  yaxis = dict(tickvals = []),
                 )
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(label="Transmitância",
                     method="update",
                     args=[{'visible': [True]*len(frames)+[False]*len(frames)}, {'yaxis':{'title': {'text':'Transmitância (u.a.)'}}},
                          ]),
                dict(label="Absorbância",
                     method="update",
                     args=[{'visible': [False]*len(frames)+[True]*len(frames)},{'yaxis':{'title': {'text':'Absorbância (u.a.)'}}},
                          ])
            ],
        )
    ]
)
fig

In [16]:
path = p/'html'/'espectrosftir.html'
fig.update_layout(newshape_line_color='grey',
                  newshape_line_dash='dash',
                  newshape_line_width=1,
                 )
fig.write_html(str(path), config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})

In [11]:
fig.update_layout(newshape_line_color='grey',
                  newshape_line_dash='dash',
                  newshape_line_width=1,
                 )
fig.write_html("ftir.html", config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})

In [12]:
fig = make_subplots(rows = 1, cols = 2,subplot_titles=("Isocianato", "Poliol"), shared_yaxes=True)
df = frames['Iso.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'green', name = 'Puro', legendgroup = 'P'), row = 1, col = 1)
df = frames['Iso-PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'red', name = 'Mistura', legendgroup = 'M'), row = 1, col = 1)
df = frames['TDI-PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'orange', name = 'Mistura aquecida', legendgroup = 'MA'), row = 1, col = 1)
df = frames['PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'blue', name = 'PAni', legendgroup = 'PA'), row = 1, col = 1)
df = frames['Poliol.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'green', showlegend = False, legendgroup = 'P'), row = 1, col = 2)
df = frames['Poliol-PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'red', showlegend = False, legendgroup = 'M'), row = 1, col = 2)
df = frames['PAni.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'blue', showlegend = False, legendgroup = 'PA'), row = 1, col = 2)
df = frames['Poliol-PAni2.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], line_color = 'orange', showlegend = False, legendgroup = 'MA'), row = 1, col = 2)
fig.update_yaxes(showticklabels = False, title_text = 'Transmitância')
fig.update_xaxes(title_text = 'Número de Onda', range = [4000,500])
fig.write_html("ftir1.html")

KeyError: 'Iso.csv'

In [ ]:
fig

In [ ]:
fig = go.Figure()
df = frames['Natural.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], name = 'Grafite Natural'))
df = frames['EG-1000.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance'], name = 'EG'))
df = frames['NP-6.csv']
fig.add_trace(go.Scatter(x = df['Wavenumber'], y = df['%Transmitance']-70, name = 'NPG'))
fig.update_yaxes(showticklabels = False, title_text = 'Transmitância (U.A.)')
fig.update_xaxes(title_text = 'Número de Onda (1/cm)', range = [4000,500])
fig.update_layout(font_size = 20, template = 'seaborn')
fig.update_traces(line_width = 1)
fig.write_html('ftir2.html')

In [ ]:
fig

## Picos

~3400: Água

1560 - 1600: N-H

In [ ]:
df = frames['Natural.csv']
df['Kind']='Natural'
dfe = frames['EG-1000.csv']
dfe['Kind']='Expandido'
dfn = frames['NP-6.csv']
dfn['Kind']='Nanoplaquetas'

In [ ]:
df = df.merge(dfe, how='outer')
df = df.merge(dfn, how='outer')

In [ ]:
mask = (df['Kind']=='Expandido') | (df['Kind']=='Nanoplaquetas')
df.loc[mask,'%Transmitance'] = df.loc[mask,'%Transmitance']-100
fig = px.line_3d(df, y = 'Wavenumber', z = '%Transmitance', x = 'Kind', color='Kind')

In [ ]:
fig.update_layout(scene_yaxis_range = [4000,400],
                  scene_xaxis_visible = False,
                  scene_yaxis_title_text = 'Número de onda (1/cm)',
                  scene_zaxis_title_text = 'Transmitância (%)',
                  legend_title_text = 'Tipo',
                  scene_yaxis_showgrid = True,
                  scene_zaxis_showgrid = False,
                  scene_zaxis_showticklabels = False,
                  scene_yaxis_showbackground = False,
                  scene_zaxis_showbackground = True,
                 )
fig.write_html('ftir3.html')

In [ ]:
fig